In [18]:
import torch
import torch.nn as nn
import pandas as pd
from nltk.tokenize import word_tokenize
from torch.utils.data import DataLoader,TensorDataset
import torch.nn.functional as f
import torch.optim as optim

In [2]:
data=pd.read_json('../Data/starwarsintents.json')

In [3]:
data=data.to_dict()

In [4]:
#create dataset
x=[]
y=[]
for k,i in enumerate(data['intents'].values()):

    for j in i['patterns']:
        x.append(j)
        y.append(k)


In [5]:
import string
vocab = [i.lower() for j in x for i in word_tokenize(j) if i not in string.punctuation]
x = [[i.lower() for i in word_tokenize(j) if i not in string.punctuation] for j in x]

In [6]:
vocab=sorted(set(vocab))
wordindx={}
onehotx=[]
for i,j in enumerate(vocab):
    wordindx[j]=i

for i in x:
    cont=[]

    for j in i:
        if j in vocab:   
            cont.append(wordindx[j])
    cont=torch.tensor(cont)
    onehotx.append(cont)


In [19]:
device=torch.device('cuda' if torch.cuda.is_available() else 'gpu')
device

device(type='cuda')

In [ ]:
y=torch.tensor(y)
x=onehotx

In [ ]:
print(len(x),y.shape)


In [10]:
torch.cuda.empty_cache()


In [11]:
class RNN(nn.Module):

    def __init__(self,embedsize,vocabsize,outputsize):
        super(RNN,self).__init__()
        self.embd=nn.Embedding(vocabsize,embedsize) #5,embedsize
        self.rnn=nn.GRU(embedsize,embedsize,2)
        self.l1=nn.Linear(embedsize,outputsize)
      
     

    def forward(self,x):
        
        x=self.embd(x)
        x,_=self.rnn(x)
        x=self.l1(x[-1,:])
        return x
    

        

In [12]:
embedsize=10
opsize=16
learningRate=0.001
epoch=300
vocabsiz=len(vocab)

In [56]:
model=RNN(embedsize,vocabsiz,opsize).to(device)


In [14]:
creteria=nn.CrossEntropyLoss()
optmzr=optim.Adam(params=model.parameters(),lr=learningRate)

In [ ]:
model.train() 
for i in range(epoch): 
    epoch_loss = 0.0
    for idx,xi in enumerate(x):
        xi=xi.to(device)
        yi=y[idx].to(device)


        output=model(xi)
        loss=creteria(output,yi)
        optmzr.zero_grad()

        loss.backward()

        optmzr.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch}/{i+1}, Loss: {epoch_loss/len(x):.4f}")

In [16]:
torch.save(model, "../model/starwars2.pth")

In [ ]:
model=torch.load("../model/starwars.pth")

In [ ]:
model.eval()
ht=None
with torch.no_grad():
    p=model(torch.tensor([99, 69,  6]).to(device))
    print(p)